In [ ]:
import os
import datetime
from datetime import date
import tushare as ts
import pandas as pd
import alphalens as al
from tqdm import tqdm

pro = ts.pro_api()
# Init today's data.
def init_today():
    # Update Factor
    if os.path.isfile("factors_df.csv"):
        print("Updating Factors Data...")
        factor_df=pd.read_csv("factors_df.csv")
        last_latest_datetime=get_last_latest_time(factor_df)
        wdays=get_wdays(last_latest_datetime,get_today())
        new_factor_df=get_factor_data(wdays)
        update_factor_df=pd.concat([factor_df, new_factor_df])
        update_factor_df.to_csv("factors_df.csv",index=False)
    else:
        print("You haven't got history data, so we are now downloading the factor data...")
        print("It might takes long.")
        wdays=get_wdays('20180101',get_today())
        factor_df=get_factor_data(wdays)
        
        
    
    if os.path.isfile("quotes_df.csv"):
        quotes_df=pd.read_csv("quotes_df.csv")
        last_latest_datetime=get_last_latest_time(quotes_df)
        nquotes=get_quotes_data(get_last_latest_time,get_today())
        update_quotes=pd.concat([quotes_df,nquotes])
        update_quotes.to_csv("quotes_df.csv",index=False)
    else:
        print("You haven't got history data, so we are now downloading the quotes data...")
        print("It might takes long.")
        nquotes=get_quotes_data('20180101',get_today())
    print("WE ARE READY FOR TODAY")
        
    
def get_last_latest_time(df):
    last_latest_datetime="20".format(
            df['trade_date'].unique().max().astype('M8[D]').astype(datetime.datetime).strftime('%y%m%d')
        )
    return last_latest_datetime

def get_wdays(start_date,end_date):
    wdays = pro.trade_cal(exchange_id='', start_date=start_date, end_date=end_date)
    wdays = wdays[wdays.is_open==1].cal_date.values.tolist()
    return wdays
        

def get_today():
    return "20{}".format(date.today().strftime('%y%m%d'))

# Transforming datetime
def transform_datetime(df):
    df['trade_date']=df['trade_date'].apply(lambda x:pd.to_datetime(x))
    return df

# Setting Factor
def set_factor(factor_df, factor):
    factor = factor_df.loc[:,['ts_code','trade_date',factor]].set_index(['trade_date','ts_code'])
    factor = factor.unstack().fillna(method='ffill').stack()
    return factor

def get_all_tickers():
    return pro.query('stock_basic', exchange='', list_status='L', fields='ts_code')['ts_code']

def get_quotes_data(start_date,end_date):
    tickers=get_all_tickers()
    quote_list=[]
    for ticker in tqdm(tickers):
        temp_quote=ts.pro_bar(ticker,start_date=start_date,end_date=end_date)
        temp_quote=temp_quote[['trade_date','close']]
        quote_list.append(temp_quote)
    quotes_df=pd.concat(quote_list)
    print("Transforming")
    quotes_df=transform_datetime(quotes_df)
    print("Saving Quotes Data...")
    quotes_df.to_csv('quotes_df.csv',index=False)
    print("Quotes Data Saved.")
    return quotes_df

def get_factor_data(wdays):
    factors_list = []
    for idate in tqdm(wdays):
        temp_df = pro.query('daily_basic', ts_code='', trade_date=idate)
        factors_list.append(temp_df)
    factor_df = pd.concat(factors_list)
    print("Transforming")
    factor_df=transform_datetime(factor_df)
    print("Saving Factor Data...")
    factor_df.to_csv('factor_df.csv',index=False)
    print("Factor Data Saved.")
    return factor_df

# Reshape for alphalens
# prices = quotes.pivot(index='trade_date',columns='ts_code',values='close')

init_today()